# Hackerrank

Julia code for solving various hackerrank problems. For reproduction purpose mostly.

## Algorithms

### Search

#### Ice cream parlor

Solved 

In [ ]:
n_trips = parse(Int, readline())

function read_trip()
    budget = parse(Int, readline())
    n = parse(Int, readline())
    prices = [parse(Int,i) for i in split(readline())]
    a,b = solve(budget, n, prices)
    println(string(min(a,b))*" "*string(max(a,b)))
end

function solve(budget, n, prices)
    for i in 1:n
        for j in (i+1):n
            if prices[i]+prices[j] == budget return (i,j) end
        end
    end
end

for i in 1:n_trips read_trip() end

#### KnightL on a Chessboard

Bruteforce approach. Work but timeout on hackerrank with first version
EDIT : Not using eval.parse reduce time from 5.6 second to 0.1 ! Worked now

In [24]:
function minimum_length(a,b,n)
    accessible, counter, finished = Set([(0,0)]), 1, false 
    while !finished
        new_accessible::Set{Tuple{Int32, Int32}} = Set()
        for old_pos in accessible
            for (f1,f2) in [(a,b),(b,a)]
                for oper1 in [+,-] for oper2 in [+,-]
                        newx::Int32 = eval(parse("$old_pos[1] $oper1 $f1"))
                        newy::Int32 = eval(parse("$old_pos[2] $oper2 $f2"))
                        if 0 ≤ newx < n && 0 ≤ newy < n 
                            new_accessible = ∪(new_accessible, Set([(newx,newy)]))
                        end end
                end
            end
        end
        if accessible == ∪(new_accessible,accessible) finished = true end
        accessible = ∪(new_accessible,accessible)
        if (n-1,n-1) in accessible 
            return counter
        end
        counter +=1
    end
    return -1
end

function main(N)
for i in 1:(N-1) 
    for j in 1:(N-1)
        print(string(minimum_length(i,j,N))*" ")
    end
    print("\n")
end
end

@time main(7)

6 4 4 8 2 12 
4 3 4 2 16 3 
4 4 2 4 4 4 
8 2 4 -1 -1 -1 
2 16 4 -1 -1 -1 
12 3 4 -1 -1 1 
  5.654091 seconds (1.04 M allocations: 55.351 MiB, 0.21% gc time)


In [29]:
add(x,y)=x+y
sub(x,y)=x-y

function minimum_length(a,b,n)
    accessible, counter, finished = Set([(0,0)]), 1, false 
    while !finished
        new_accessible::Set{Tuple{Int32, Int32}} = Set()
        for old_pos in accessible
            for (f1,f2) in [(a,b),(b,a)]
                for oper1 in [add,sub] for oper2 in [add,sub]
                        newx::Int32 = oper1(old_pos[1],f1)
                        newy::Int32 = oper2(old_pos[2],f2)
                        if 0 ≤ newx < n && 0 ≤ newy < n 
                            new_accessible = ∪(new_accessible, Set([(newx,newy)]))
                        end end
                end
            end
        end
        if accessible == ∪(new_accessible,accessible) finished = true end
        accessible = ∪(new_accessible,accessible)
        if (n-1,n-1) in accessible 
            return counter
        end
        counter +=1
    end
    return -1
end

function main(N)
for i in 1:(N-1) 
    for j in 1:(N-1)
        print(string(minimum_length(i,j,N))*" ")
    end
    print("\n")
end
end

@time main(7)

6 4 4 8 2 12 
4 3 4 2 16 3 
4 4 2 4 4 4 
8 2 4 -1 -1 -1 
2 16 4 -1 -1 -1 
12 3 4 -1 -1 1 
  0.121124 seconds (117.65 k allocations: 10.278 MiB, 3.82% gc time)
